In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import TargetEncoder


# 1. Phân tích dữ liệu
## 1.1 Mô tả dữ liệu


Dữ liệu cho mô hình dự đoán lương `data/salaries.csv`. Dữ liệu bao gồm 11 cột và 16495 hàng dự liệu bao gồm:

1. `work_year`: năm làm việc

2. `experience_level`: Trình độ (`EN`:Entry-level / Junior; `MI`: Mid-MI: Mid-level / Intermediate `SE` : Senior-level / Expert `EX`: Executive-level / Director)

3. `employment_type`: Loại công việc (`PT`: Part-time `FT`: Full-time `CT`: Contract `FL`: Freelance)

4. `job_title`: Vai trò đảm nhiệm

5. `salary` : Mức lương

6. `salary_currency`: Đơn vị tiền của tiền lương được trả dưới dạng mã tiền tệ ISO 4217.

7. `salary_in_usd`: Lương bằng USD

8. `employee_residence`: Quốc gia cư trú chính của nhân viên trong năm làm việc dưới dạng mã quốc gia ISO 3166.

9. `remote_ratio` : Tổng khối lượng công việc được thực hiện từ xa, các giá trị có thể có như sau : `0`: No remote work (less than 20%)
`50`: Partially remote
`100`: Fully remote (more than 80%)

10. `company_location`: Quốc gia của văn phòng chính hoặc chi nhánh hợp đồng của người sử dụng lao động là mã quốc gia ISO 3166.

11. `company_size` : Quy mô công ty ( `S`: less than 50 employees (small)
`M`: 50 to 250 employees (medium)
`L`: more than 250 employees (large) )

In [2]:
df = pd.read_csv('data/salaries.csv')
print(df.shape)
print(df.describe())
df.head()

(16494, 11)
          work_year        salary  salary_in_usd  remote_ratio
count  16494.000000  1.649400e+04   16494.000000  16494.000000
mean    2023.224991  1.637878e+05  149713.575725     32.044986
std        0.713405  3.406017e+05   68516.136918     46.260201
min     2020.000000  1.400000e+04   15000.000000      0.000000
25%     2023.000000  1.020000e+05  101517.500000      0.000000
50%     2023.000000  1.422000e+05  141300.000000      0.000000
75%     2024.000000  1.873422e+05  185900.000000    100.000000
max     2024.000000  3.040000e+07  800000.000000    100.000000


,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2024,MI,FT,Data Scientist,120000,USD,120000,AU,0,AU,S
1,2024,MI,FT,Data Scientist,70000,USD,70000,AU,0,AU,S
2,2024,MI,CT,Data Scientist,130000,USD,130000,US,0,US,M
3,2024,MI,CT,Data Scientist,110000,USD,110000,US,0,US,M
4,2024,MI,FT,Data Science Manager,240000,USD,240000,US,0,US,M


In [3]:
df.columns

Index(['work_year', 'experience_level', 'employment_type', 'job_title',
       'salary', 'salary_currency', 'salary_in_usd', 'employee_residence',
       'remote_ratio', 'company_location', 'company_size'],
      dtype='object')

In [4]:
df['experience_level'].unique()

array(['MI', 'SE', 'EN', 'EX'], dtype=object)

In [5]:
df

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2024,MI,FT,Data Scientist,120000,USD,120000,AU,0,AU,S
1,2024,MI,FT,Data Scientist,70000,USD,70000,AU,0,AU,S
2,2024,MI,CT,Data Scientist,130000,USD,130000,US,0,US,M
3,2024,MI,CT,Data Scientist,110000,USD,110000,US,0,US,M
4,2024,MI,FT,Data Science Manager,240000,USD,240000,US,0,US,M
...,...,...,...,...,...,...,...,...,...,...,...
16489,2020,SE,FT,Data Scientist,412000,USD,412000,US,100,US,L
16490,2021,MI,FT,Principal Data Scientist,151000,USD,151000,US,100,US,L
16491,2020,EN,FT,Data Scientist,105000,USD,105000,US,100,US,S
16492,2020,EN,CT,Business Data Analyst,100000,USD,100000,US,100,US,L


In [6]:
# xem thông tin dữ liệu 
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16494 entries, 0 to 16493
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   work_year           16494 non-null  int64 
 1   experience_level    16494 non-null  object
 2   employment_type     16494 non-null  object
 3   job_title           16494 non-null  object
 4   salary              16494 non-null  int64 
 5   salary_currency     16494 non-null  object
 6   salary_in_usd       16494 non-null  int64 
 7   employee_residence  16494 non-null  object
 8   remote_ratio        16494 non-null  int64 
 9   company_location    16494 non-null  object
 10  company_size        16494 non-null  object
dtypes: int64(4), object(7)
memory usage: 1.4+ MB
None


## 1.2 Tiền xử lý

In [7]:
df.dtypes # Kiểu dữ liệu của từng cột

work_year              int64
experience_level      object
employment_type       object
job_title             object
salary                 int64
salary_currency       object
salary_in_usd          int64
employee_residence    object
remote_ratio           int64
company_location      object
company_size          object
dtype: object

In [8]:
df.isna().sum() # kiểm tra xem số dữ liệu rỗng

work_year             0
experience_level      0
employment_type       0
job_title             0
salary                0
salary_currency       0
salary_in_usd         0
employee_residence    0
remote_ratio          0
company_location      0
company_size          0
dtype: int64

### Chuẩn hóa dữ liệu 
Do có nhiều dữ liệu dạng object vì vậy ta cần chuẩn hóa phù hợp để đưa vào mô hình

Chuẩn hóa dữ liệu dạng Label Encoding cho các cột có thứ tự như  `experience_level` và `company_size`

In [9]:
print(df['experience_level'].unique())
experience = {'MI': 1, 'SE': 2, 'EX': 3, 'EN': 4}
df['experience_level_nomr'] = df['experience_level'].map(experience)
company_size_nomr = {'S': 1,'M': 2,'L': 3,'S ': 1}
df['company_size_nomr'] = df['company_size'].map(company_size_nomr)
df['company_size_nomr'] = df['company_size_nomr'].astype(int)
df = df.drop(columns=['experience_level','company_size'])
df
# Chuẩn hóa dữ liệu 

['MI' 'SE' 'EN' 'EX']


,work_year,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,experience_level_nomr,company_size_nomr
0,2024,FT,Data Scientist,120000,USD,120000,AU,0,AU,1,1
1,2024,FT,Data Scientist,70000,USD,70000,AU,0,AU,1,1
2,2024,CT,Data Scientist,130000,USD,130000,US,0,US,1,2
3,2024,CT,Data Scientist,110000,USD,110000,US,0,US,1,2
4,2024,FT,Data Science Manager,240000,USD,240000,US,0,US,1,2
...,...,...,...,...,...,...,...,...,...,...,...
16489,2020,FT,Data Scientist,412000,USD,412000,US,100,US,2,3
16490,2021,FT,Principal Data Scientist,151000,USD,151000,US,100,US,1,3
16491,2020,FT,Data Scientist,105000,USD,105000,US,100,US,4,1
16492,2020,CT,Business Data Analyst,100000,USD,100000,US,100,US,4,3


In [10]:
# ta lấy chuẩn đơn vị lương tính theo đơn vị nghìn USD loại bỏ các feature không cần thiết
df = df.drop(columns=['salary_currency','salary','employee_residence'])
df['salary_in_usd']  = df['salary_in_usd'] / 1000
df

,work_year,employment_type,job_title,salary_in_usd,remote_ratio,company_location,experience_level_nomr,company_size_nomr
0,2024,FT,Data Scientist,120.000,0,AU,1,1
1,2024,FT,Data Scientist,70.000,0,AU,1,1
2,2024,CT,Data Scientist,130.000,0,US,1,2
3,2024,CT,Data Scientist,110.000,0,US,1,2
4,2024,FT,Data Science Manager,240.000,0,US,1,2
...,...,...,...,...,...,...,...,...
16489,2020,FT,Data Scientist,412.000,100,US,2,3
16490,2021,FT,Principal Data Scientist,151.000,100,US,1,3
16491,2020,FT,Data Scientist,105.000,100,US,4,1
16492,2020,CT,Business Data Analyst,100.000,100,US,4,3


Chuẩn hóa  One-Hot Encoding cho các dữ liệu phân loại

In [11]:
df = pd.get_dummies(df)
df 

,work_year,salary_in_usd,remote_ratio,experience_level_nomr,company_size_nomr,employment_type_CT,employment_type_FL,employment_type_FT,employment_type_PT,job_title_AI Architect,...,company_location_SA,company_location_SE,company_location_SG,company_location_SI,company_location_TH,company_location_TR,company_location_UA,company_location_US,company_location_VN,company_location_ZA
0,2024,120.000,0,1,1,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
1,2024,70.000,0,1,1,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
2,2024,130.000,0,1,2,True,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
3,2024,110.000,0,1,2,True,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
4,2024,240.000,0,1,2,False,False,True,False,False,...,False,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16489,2020,412.000,100,2,3,False,False,True,False,False,...,False,False,False,False,False,False,False,True,False,False
16490,2021,151.000,100,1,3,False,False,True,False,False,...,False,False,False,False,False,False,False,True,False,False
16491,2020,105.000,100,4,1,False,False,True,False,False,...,False,False,False,False,False,False,False,True,False,False
16492,2020,100.000,100,4,3,True,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False


In [12]:
df

,work_year,salary_in_usd,remote_ratio,experience_level_nomr,company_size_nomr,employment_type_CT,employment_type_FL,employment_type_FT,employment_type_PT,job_title_AI Architect,...,company_location_SA,company_location_SE,company_location_SG,company_location_SI,company_location_TH,company_location_TR,company_location_UA,company_location_US,company_location_VN,company_location_ZA
0,2024,120.000,0,1,1,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
1,2024,70.000,0,1,1,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
2,2024,130.000,0,1,2,True,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
3,2024,110.000,0,1,2,True,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
4,2024,240.000,0,1,2,False,False,True,False,False,...,False,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16489,2020,412.000,100,2,3,False,False,True,False,False,...,False,False,False,False,False,False,False,True,False,False
16490,2021,151.000,100,1,3,False,False,True,False,False,...,False,False,False,False,False,False,False,True,False,False
16491,2020,105.000,100,4,1,False,False,True,False,False,...,False,False,False,False,False,False,False,True,False,False
16492,2020,100.000,100,4,3,True,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False


In [13]:
df.to_csv('data/salaries_norm.csv',index=False) # lưu lại file đã xử lý